# pandas的拼接操作

pandas的拼接分为两种：
- 级联：pd.concat, pd.append
- 合并：pd.merge, pd.join

## 1. 使用pd.concat()级联

pandas使用pd.concat函数，与np.concatenate函数类似，只是多了一些参数：
```
objs
axis=0
keys
join='outer' / 'inner':表示的是级联的方式，outer会将所有的项进行级联（忽略匹配和不匹配），而inner只会将匹配的项级联到一起，不匹配的不级联
ignore_index=False
```

### 1)匹配级联

In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

In [2]:
df1 = DataFrame(data=np.random.randint(0,100,size=(3,3)),index=['a','b','c'],columns=['A','B','C'])
df2 = DataFrame(data=np.random.randint(0,100,size=(3,3)),index=['a','d','c'],columns=['A','d','C'])

In [7]:
pd.concat((df1,df1),axis=0,join='inner')

,A,B,C
a,59,40,89
b,71,5,76
c,29,34,87
a,59,40,89
b,71,5,76
c,29,34,87


### 2) 不匹配级联

不匹配指的是级联的维度的索引不一致。例如纵向级联时列索引不一致，横向级联时行索引不一致

有2种连接方式：

- 外连接：补NaN（默认模式）

- 内连接：只连接匹配的项

In [11]:
pd.concat((df1,df2),axis=1,join='outer')

C:\jupyter\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,A,B,C,A,d,C
a,59.0,40.0,89.0,50.0,26.0,45.0
b,71.0,5.0,76.0,NaN,NaN,NaN
c,29.0,34.0,87.0,31.0,82.0,35.0
d,NaN,NaN,NaN,23.0,95.0,94.0


### 3) 使用df.append()函数添加

由于在后面级联的使用非常普遍，因此有一个函数append专门用于在后面添加

## 2. 使用pd.merge()合并

merge与concat的区别在于，merge需要依据某一共同的列来进行合并

使用pd.merge()合并时，会自动根据两者相同column名称的那一列，作为key来进行合并。

注意每一列元素的顺序不要求一致

参数：
- how：out取并集   inner取交集

- on：当有多列相同的时候，可以使用on来指定使用那一列进行合并，on的值为一个列表

###  1) 一对一合并

In [12]:
df1 = DataFrame({'employee':['Bob','Jake','Lisa'],
                'group':['Accounting','Engineering','Engineering'],
                })
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering


In [13]:
df2 = DataFrame({'employee':['Lisa','Bob','Jake'],
                'hire_date':[2004,2008,2012],
                })
df2

,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012


In [14]:
pd.merge(df1,df2,how='outer')

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004


### 2) 多对一合并

In [15]:
df3 = DataFrame({
    'employee':['Lisa','Jake'],
    'group':['Accounting','Engineering'],
    'hire_date':[2004,2016]})
df3

,employee,group,hire_date
0,Lisa,Accounting,2004
1,Jake,Engineering,2016


In [16]:
df4 = DataFrame({'group':['Accounting','Engineering','Engineering'],
                       'supervisor':['Carly','Guido','Steve']
                })
df4

,group,supervisor
0,Accounting,Carly
1,Engineering,Guido
2,Engineering,Steve


In [17]:
pd.merge(df3,df4)

,employee,group,hire_date,supervisor
0,Lisa,Accounting,2004,Carly
1,Jake,Engineering,2016,Guido
2,Jake,Engineering,2016,Steve


### 3) 多对多合并

In [18]:
df1 = DataFrame({'employee':['Bob','Jake','Lisa'],
                 'group':['Accounting','Engineering','Engineering']})
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering


In [19]:
df5 = DataFrame({'group':['Engineering','Engineering','HR'],
                'supervisor':['Carly','Guido','Steve']
                })
df5

,group,supervisor
0,Engineering,Carly
1,Engineering,Guido
2,HR,Steve


In [21]:
pd.merge(df1,df5,how='outer')

,employee,group,supervisor
0,Bob,Accounting,NaN
1,Jake,Engineering,Carly
2,Jake,Engineering,Guido
3,Lisa,Engineering,Carly
4,Lisa,Engineering,Guido
5,NaN,HR,Steve


- 加载excl数据:pd.read_excel('excl_path',sheetname=1)

### 4) key的规范化

- 当列冲突时，即有多个列名称相同时，需要使用on=来指定哪一个列作为key，配合suffixes指定冲突列名

In [10]:
df1 = DataFrame({'employee':['Jack',"Summer","Steve"],
                 'group':['Accounting','Finance','Marketing']})

In [11]:
df2 = DataFrame({'employee':['Jack','Bob',"Jake"],
                 'hire_date':[2003,2009,2012],
                'group':['Accounting','sell','ceo']})

In [22]:
display(df1,df2)

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering


,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012


   - 当两张表没有可进行连接的列时，可使用left_on和right_on手动指定merge中左右两边的哪一列列作为连接的列

In [12]:
df1 = DataFrame({'employee':['Bobs','Linda','Bill'],
                'group':['Accounting','Product','Marketing'],
               'hire_date':[1998,2017,2018]})

In [13]:
df5 = DataFrame({'name':['Lisa','Bobs','Bill'],
                'hire_dates':[1998,2016,2007]})

In [23]:
display(df1,df5)

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering


,group,supervisor
0,Engineering,Carly
1,Engineering,Guido
2,HR,Steve


============================================

练习：


1. 自行练习多对一，多对多的情况  

2. 自学left_index,right_index

============================================

### 5) 内合并与外合并:out取并集   inner取交集

- 内合并：只保留两者都有的key（默认模式）

In [25]:
df6 = DataFrame({'name':['Peter','Paul','Mary'],
               'food':['fish','beans','bread']}
               )
df7 = DataFrame({'name':['Mary','Joseph'],
                'drink':['wine','beer']})


In [26]:
display(df6,df7)

,name,food
0,Peter,fish
1,Paul,beans
2,Mary,bread


,name,drink
0,Mary,wine
1,Joseph,beer


- 外合并 how='outer'：补NaN

In [27]:
df6 = DataFrame({'name':['Peter','Paul','Mary'],
               'food':['fish','beans','bread']}
               )
df7 = DataFrame({'name':['Mary','Joseph'],
                'drink':['wine','beer']})
display(df6,df7)
pd.merge()

,name,food
0,Peter,fish
1,Paul,beans
2,Mary,bread


,name,drink
0,Mary,wine
1,Joseph,beer


## 作业
## 3. 案例分析：美国各州人口数据分析